In [4]:
import sys, os, glob
from copy import deepcopy

import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F
import torch.optim

import MDAnalysis as mda
import warnings
warnings.filterwarnings("ignore")

import modeller
from modeller import *
from modeller.scripts import complete_pdb

import biobox as bb

#edit path as required for your computer (or remove, if you installed molearn via conda-forge)
sys.path.insert(0, '/home/rzhu/Desktop/projects/TPS/molearn/src')
import molearn
from molearn import load_data, Auto_potential
from molearn.autoencoder import Autoencoder

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

ModuleNotFoundError: No module named 'modeller'